In [5]:
from google.colab import drive
drive.mount('/content/drive')

# # 2021 Cancer
# ## 使用者輸入
# main_file_name = "2023Report"
# year = "2021"
year = str((input("請輸入年份 year（i.e. 2021）：")))
main_file_name = str(input(f"請輸入 {year}_Cancer 所在的資料夾名稱（請記得先將 {year}_Cancer.csv 檔案放置在此資料夾）(i.e. 2023Report)"))

import os
directory_path = f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}'
os.makedirs(directory_path, exist_ok=True)
if os.path.exists(directory_path):
    print(f"目錄 '{directory_path}' 已成功建立！")
else:
    print(f"目錄 '{directory_path}' 建立失敗！")


################################################################################################################################################################
def clean_data(df):
    df['site'] = df['site'].str.replace("'", '')
    df['site-c'] = df['site'].str.replace("C", '').astype(int)
    df['hist/behavior'] = df['hist/behavior'].str.replace("'", '').astype(int)
    df['診斷年齡'] = df['age'].str.replace("'", '').astype(int)
    df['性別'] = "9"
    df['性別'] = df['sex'].str.replace("'1", '男')
    df['性別'] = df['性別'].str.replace("'2", '女')
    df['個案分類'] = "9"
    df['個案分類'] = df['class'].str.replace("'1", 'class1')
    df['個案分類'] = df['個案分類'].str.replace("'2", 'class2')
    df['個案分類'] = df['個案分類'].str.replace("'3", 'class3')
    df['個案分類'] = df['個案分類'].str.replace("'0", 'class0')
    df['Site'] = 1
    return df


####
def Site_Group_InLymphoma(site):
    if (769< site <780):
        return ("淋巴結(內)惡性淋巴癌")

def Hist_Behavior_Group(hist_):
    if ((96499< hist_ < 96680) or (96699 < hist_ < 97000) or (97609 < hist_ < 97620) or (98259 < hist_ < 98270) or (98329 < hist_ < 98340) or (99399 <hist_ < 99410) or (95959 <hist_ < 95970) or (96999 < hist_ < 97300) or (98269 < hist_ < 98280) or (97499 < hist_ < 97610) or (97639 < hist_ < 97650) or (95899 < hist_ < 95920)):
        return ("淋巴結(外)惡性淋巴瘤")

def Site_Group(site):
    if ((site < 10) or (19 <site < 24) or (27 <site < 32) or (38 <site < 42) or (47 <site < 51) or (57 <site < 63) or (67 <site < 70)):
        return ("口腔癌")
    if ((site == 19) or (site == 24) or ( 50 < site < 53) or (89 < site < 101) or (site == 104) or (107 < site < 110) or (site == 142) or (site == 148)):
        return ("口咽癌")
    if ((128 < site < 133) or (137 < site < 141)):
        return ("下咽癌")
    if ((319 < site < 330)):
        return ("喉癌")
    if (109 < site < 120):
        return ("鼻咽癌")
    if ((149 < site < 156) or (157< site < 160)):
        return ("食道癌")
    if ((159< site < 167) or (167 < site < 170)):
        return ("胃癌")
    if ((179 <site <190)):
        return ("結腸癌")
    if ((site == 199) or (site == 209)):
        return ("直腸癌")
    if (209< site < 220):
        return ("肛門癌")
    if (219< site <222):
        return ("肝癌")
    if ((249< site < 255) or (256< site< 260)):
        return ("胰臟癌")
    if ((339< site < 344) or (347 < site < 350)):
        return ("肺癌")
    if (499< site < 510):
        return ("乳癌")
    if ((529< site <532) or (537< site < 540)):
        return ("子宮頸癌")
    if ((539< site < 544) or (site == 549) or (site ==559)):
        return ("子宮體癌")
    if (site == 569):
        return ("卵巢癌")
    if (site == 619):
        return ("攝護腺癌")
    if (669< site <680):
        return ("膀胱癌")
    # if (769< site <780):
    #     return ("淋巴結(內)惡性淋巴癌")
    if (site == 421):
        return ("白血病")

    return ("其他（待確認）")

####
def Save_separated_file(df):
    cancer_types = df['Site'].unique()
    for cancer_type in cancer_types:
        subset = df[df['Site'] == cancer_type].reset_index(drop=True)
        directory_path = f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{cancer_type}'
        os.makedirs(directory_path, exist_ok=True)
        excel_file_name = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{cancer_type}/{year}{cancer_type}.xlsx"
        csv_file_name = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{cancer_type}/{year}{cancer_type}.csv"
        subset.to_excel(excel_file_name, index=False)
        subset.to_csv(csv_file_name, index=False)

def separate_class_of_carcinoma(df):
    df['Site'] = df['site-c'].apply(Site_Group_InLymphoma)
    condition = ((df['Site'] == '淋巴結(內)惡性淋巴癌'))
    in_lymphoma_df = df[condition]
    df.drop(df[condition].index, inplace=True)
    in_lymphoma_df = in_lymphoma_df.reset_index(drop=True)
    df = df.reset_index(drop=True)

    df['Site'] = df['hist/behavior'].apply(Hist_Behavior_Group)
    condition = ((df['Site'] == '淋巴結(外)惡性淋巴瘤'))
    ex_lymphoma_df = df[condition]
    df.drop(df[condition].index, inplace=True)
    ex_lymphoma_df = ex_lymphoma_df.reset_index(drop=True)
    df = df.reset_index(drop=True)                              # lymphoma_df, df

    df['Site'] = df['site-c'].apply(Site_Group)
    merged_df = pd.concat([in_lymphoma_df, ex_lymphoma_df], ignore_index=True)
    merged_df = pd.concat([merged_df, df], ignore_index=True)
    df = merged_df.copy()
    df = df.reset_index(drop=True)
    Save_separated_file(df)
    return df
################################################################################################################################################################


import pandas as pd
# file_path = f"./{main_file_name}/{year}_Cancer.csv"
file_path = f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/{year}_Cancer.csv'
df = pd.read_csv(file_path)
df = clean_data(df)
df = separate_class_of_carcinoma(df)

# import matplotlib.pyplot as plt
Site_distribution = df['Site'].value_counts()
print(Site_distribution)
# Site_distribution.plot(kind='bar', color='skyblue', edgecolor='black')
# plt.title('Site Group Distribution')
# plt.xlabel('Site Group')
# plt.ylabel('Count')
# plt.show()


# # 各癌別

def Age_Group(age):
    bins = [40, 50, 60, 70, 80]
    labels = ['≦39', '40-49', '50-59', '60-69', '70-79', '≧80']

    for i in range(len(bins)):
        if age < bins[i]:
            return labels[i]

    return labels[-1]

def Age_Gender_Report(df):
    df_report = df[['診斷年齡', '性別']]
    df_report['年齡區間'] = df_report['診斷年齡'].apply(Age_Group)
    # df_report
    order = ['≦39', '40-49', '50-59', '60-69', '70-79', '≧80']
    cross_table = pd.crosstab(df_report['性別'], df_report['年齡區間'], margins=True, margins_name='總計')
    cross_table = cross_table.reindex(columns=order)
    cross_table['總計'] = cross_table.sum(axis=1)
    # cross_table
    percentage_table = cross_table.div(cross_table['總計'], axis=0) * 100
    percentage_table = percentage_table.round(1)
    # percentage_table
    total_percentage = [percentage_table.xs('總計')]
    # list(total_percentage[0])
    cross_table.loc['總計百分比'] = list(total_percentage[0])
    return cross_table, df_report

####
def Class_Gender_Report(df):
    df_report = df[['個案分類', '性別']]
    # df_report
    order = ['class0', 'class1', 'class2', 'class3']
    cross_table = pd.crosstab(df_report['性別'], df_report['個案分類'], margins=True, margins_name='總計')
    cross_table = cross_table.reindex(columns=order)
    cross_table['總計'] = cross_table.sum(axis=1)
    # Fill missing values with 0
    cross_table = cross_table.fillna(0)
    # cross_table
    percentage_table = cross_table.div(cross_table['總計'], axis=0) * 100
    percentage_table = percentage_table.round(1)
    # percentage_table
    total_percentage = [percentage_table.xs('總計')]
    # list(total_percentage[0])
    cross_table.loc['總計百分比'] = list(total_percentage[0])
    return cross_table, df_report


####
def clean(df):
    df_AJCC_stage = df[['class', 'pdescr', 'pstage', 'cstage']]
    df_AJCC_stage['class'] = df_AJCC_stage['class'].str.replace("'", '').astype(int)
    df_AJCC_stage = df_AJCC_stage[df_AJCC_stage['class'].isin([1, 2])]
    df_AJCC_stage = df_AJCC_stage.reset_index(drop=True)
    df_AJCC_stage['pdescr'] = df_AJCC_stage['pdescr'].str.replace("'", '').astype(int)
    df_AJCC_stage['pstage'] = df_AJCC_stage['pstage'].str.replace("'", '') #str
    df_AJCC_stage['cstage'] = df_AJCC_stage['cstage'].str.replace("'", '') #str
    return df_AJCC_stage


def to_define_stage(df_AJCC_stage, i):
    if (df_AJCC_stage['pdescr'][i] in [4, 6]):
        return df_AJCC_stage['cstage'][i]
    if (df_AJCC_stage['pstage'][i] in ["999", "888", "BBB"]):
        return df_AJCC_stage['cstage'][i]
    return df_AJCC_stage['pstage'][i]

def Define_Stage_(df):
    df_AJCC_stage = clean(df)
    df_AJCC_stage['STAGE'] = None
    for i in range (len(df_AJCC_stage)):
        df_AJCC_stage['STAGE'][i] = to_define_stage(df_AJCC_stage, i)
    return df_AJCC_stage


def Distribution_tabel(order, df_AJCC_stage):
    _distribution = df_AJCC_stage['STAGE'].value_counts()
    _distribution = df_AJCC_stage.groupby('STAGE').size().reindex(order, fill_value=0).reset_index(name='count')
    sum_counts = _distribution.loc[_distribution['STAGE'].isin(['BBB', '888', '999']), 'count'].sum()
    _distribution = _distribution[~_distribution['STAGE'].isin(['BBB', '888', '999'])]
    new_row = pd.DataFrame([{'STAGE': '不詳期別', 'count': sum_counts}])
    _distribution = pd.concat([_distribution, new_row], ignore_index=True)
    _distribution.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_AJCC_stage.xlsx', sheet_name='AJCC_stage')
    return _distribution


################################################################################################################################################################
def generate_and_save_reports(df, carcinoma_type):
    directory_path = f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report'
    os.makedirs(directory_path, exist_ok=True)

    age_gender_report, df_report = Age_Gender_Report(df)
    #Save file
    age_gender_report.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_Age_Gender.xlsx', sheet_name='age_gender_report')

    class_gender_report, df_report = Class_Gender_Report(df)
    #Save file
    class_gender_report.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_Class_Gender.xlsx', sheet_name='class_gender_report')


    df_AJCC_stage = Define_Stage_(df)
    #Save file
    df_AJCC_stage.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_AJCC_stage.xlsx', sheet_name='AJCC_stage')


    if carcinoma_type == "白血病":
        return age_gender_report, class_gender_report

    _distribution = Type_Report(df)
    # Save file
    _distribution.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_Type(Class1-2).xlsx', sheet_name='class_type')

    return age_gender_report, class_gender_report, _distribution, df_AJCC_stage
################################################################################################################################################################

"""
### 頭頸癌治療順序
"""

def therapy_order_Head_Neck_(df):
    #### 資料選取
    therapy_order = ['class', 'optype_o', 'optype_h', 'rtmodal', 'ort_modal', 'srs', 'sls', 'chem_h', 'horm_h', 'immu_h', 'htep_h', 'targe_Tfacility', 'palli_h', 'other_T']
    therapy_df = df[therapy_order]
    for i in therapy_order:
        therapy_df[i] = therapy_df[i].str.replace(r"[^0-9]", '', regex=True)
        therapy_df[i] = pd.to_numeric(therapy_df[i], errors='coerce')
    therapy_df = therapy_df[therapy_df['class'].isin([1, 2])]
    therapy_df = therapy_df.reset_index()#.drop('index', axis=1)
    # therapy_df

    ########################################## "口腔癌\口咽癌\下咽癌\喉癌\鼻咽癌「治療」定義 " ##########################################
    # OP
    OP_df = therapy_df[~((therapy_df['optype_o'].isin([0, 99])) & (therapy_df['optype_h'].isin([0, 99])))]
    OP_df = OP_df.reset_index()#.drop('index', axis=1)
    OP_list = list(OP_df['index'])
    op = set(OP_list)
    # OP_list

    # RT
    RT_df = therapy_df[(therapy_df['rtmodal'] >= 1) & (therapy_df['rtmodal'] <= 64)]
    RT_df = RT_df.reset_index()#.drop('index', axis=1)
    RT_list = list(RT_df['index'])
    rt = set(RT_list)
    # RT_list


    # CHEM
    CHEM_df = therapy_df[(therapy_df['chem_h'] >= 1) & (therapy_df['chem_h'] <= 31)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem = set(CHEM_list)
    # CHEM_list

    # # HORM
    # HORM_df = therapy_df[(therapy_df['horm_h'] >= 1) & (therapy_df['horm_h'] <= 31)]
    # HORM_df = HORM_df.reset_index()#.drop('index', axis=1)
    # HORM_list = list(HORM_df['index'])
    # horm = set(HORM_list)
    # HORM_list

    # # IMMU
    # IMMU_df = therapy_df[(therapy_df['immu_h'] >= 10) & (therapy_df['immu_h'] <= 50)]
    # IMMU_df = IMMU_df.reset_index()#.drop('index', axis=1)
    # IMMU_list = list(IMMU_df['index'])
    # immu = set(IMMU_list)
    # IMMU_list

    # # HTEP
    # HTEP_df = therapy_df[(therapy_df['htep_h'] >= 1) & (therapy_df['htep_h'] <= 31)]
    # HTEP_df = HTEP_df.reset_index()#.drop('index', axis=1)
    # HTEP_list = list(HTEP_df['index'])
    # htep = set(HTEP_list)
    # HTEP_list

    # TARGE
    TARGE_df = therapy_df[(therapy_df['targe_Tfacility'] >= 1) & (therapy_df['targe_Tfacility'] <= 31)]
    TARGE_df = TARGE_df.reset_index()#.drop('index', axis=1)
    TARGE_list = list(TARGE_df['index'])
    tg = set(TARGE_list)
    # TARGE_list

    # PALLI
    PALLI_df = therapy_df[(therapy_df['palli_h'] >= 1) & (therapy_df['palli_h'] <= 7)]
    PALLI_df = PALLI_df.reset_index()#.drop('index', axis=1)
    PALLI_list = list(PALLI_df['index'])
    palli_therapy = set(PALLI_list)
    # PALLI_list


    # Other
    Other_df = therapy_df[(therapy_df['other_T'] >= 1) & (therapy_df['other_T'] <= 3)]
    Other_df = Other_df.reset_index()#.drop('index', axis=1)
    Other_list = list(Other_df['index'])
    other_therapy = set(Other_list)
    # Other_list

    # 同步(~非同步~)化療與放療或/與標靶治療
    sls_df = therapy_df[(therapy_df['sls'].isin([2,6]))]
    sls_df = sls_df.reset_index()#.drop('index', axis=1)
    # sls_df
    sls_list = list(sls_df['index'])
    sls = set(sls_list)
    # sls_list
    ########################################## "口腔癌\口咽癌\下咽癌\喉癌\鼻咽癌「治療」定義 end" ##########################################

    ########################################## "口腔癌\口咽癌\下咽癌\喉癌\鼻咽癌「治療」範圍" ##########################################
    #### 狄摩根規則
    result_dict = {
        "治療方式": [],
        "人數": [],
        f"{carcinoma_type} 病患 Index List": []
    }


    # 手術
    OP_therapy = list(op - (rt | chem))
    # print(f"OP_therapy:\t\t\t\t\t\t{OP_therapy}")
    result_dict["治療方式"].append("手術")
    result_dict["人數"].append(len(OP_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_therapy if OP_therapy else None)


    # 手術合併化療與放療
    OP_RT_CHEM_combined_therapy = list(op & rt & chem)
    # print(f"OP_RT_CHEM_combined_therapy:\t\t\t\t{OP_RT_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療與放療")
    result_dict["人數"].append(len(OP_RT_CHEM_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_RT_CHEM_combined_therapy if OP_RT_CHEM_combined_therapy else None)


    # 手術合併放療
    OP_RT_combined_therapy = list(set(op & rt) - set(OP_RT_CHEM_combined_therapy))
    # print(f"OP_RT_combined_therapy:\t\t\t\t\t{OP_RT_combined_therapy}")
    result_dict["治療方式"].append("手術合併放療")
    result_dict["人數"].append(len(OP_RT_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_RT_combined_therapy if OP_RT_combined_therapy else None)


    # 手術合併化療
    OP_CHEM_combined_therapy = list(set(op & chem) - set(OP_RT_CHEM_combined_therapy))
    # print(f"OP_CHEM_combined_therapy:\t\t\t\t{OP_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療")
    result_dict["人數"].append(len(OP_CHEM_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_combined_therapy if OP_CHEM_combined_therapy else None)


    # （非）同步 化療與放療或/與標靶治療
    chem_rt_target_indexes = set(set(chem & rt) - set(OP_RT_CHEM_combined_therapy))
    # print(f"chem_rt_target_: {chem_rt_target_indexes}")
    sync_chem_rt_target_therapy = []   # 同步化療與放療或/與標靶治療 == sls
    async_chem_rt_target_therapy = []  # 非同步化療與放療或/與標靶治療
    for i in chem_rt_target_indexes:
        if i in sls:
            sync_chem_rt_target_therapy.append(i)
        else:
            async_chem_rt_target_therapy.append(i)
    # print(f"sync_chem_rt_target_therapy:\t\t\t\t{sync_chem_rt_target_therapy}\nasync_chem_rt_target_therapy:\t\t\t\t{async_chem_rt_target_therapy}")
    result_dict["治療方式"].append("同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(sync_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(sync_chem_rt_target_therapy if sync_chem_rt_target_therapy else None)


    result_dict["治療方式"].append("非同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(async_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(async_chem_rt_target_therapy if async_chem_rt_target_therapy else None)



    # 放療
    RT_therapy = list(rt - (op | chem))
    # print(f"RT_therapy:\t\t\t\t\t\t{RT_therapy}")
    result_dict["治療方式"].append("放療")
    result_dict["人數"].append(len(RT_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(RT_therapy if RT_therapy else None)



    # 化療或/與標靶治療
    chem_therapy = list(chem - (op | rt))
    # print(f"chem_therapy:\t\t\t\t\t\t{chem_therapy}")
    result_dict["治療方式"].append("化療或/與標靶治療")
    result_dict["人數"].append(len(chem_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(chem_therapy if chem_therapy else None)


    # 緩和治療
    palli_therapy = list(palli_therapy)
    # print(f"palli_therapy:\t\t\t\t\t\t{palli_therapy}")
    result_dict["治療方式"].append("緩和治療")
    result_dict["人數"].append(len(palli_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(palli_therapy if palli_therapy else None)


    # 其他治療
    other_therapy = list(other_therapy)
    # print(f"other_therapy:\t\t\t\t\t\t{other_therapy}")
    result_dict["治療方式"].append("其他治療")
    result_dict["人數"].append(len(other_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(other_therapy if other_therapy else None)

    # result_dict
    result_df = pd.DataFrame(result_dict)
    # result_df

    #Save file
    result_df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_therapy_type.xlsx', sheet_name='therapy')
    ########################################## "口腔癌\口咽癌\下咽癌\喉癌\鼻咽癌「治療」範圍 end" ##########################################
    return result_df

"""# carcinoma_type = "OralCavity"
"""

carcinoma_type = "口腔癌"

def Type_Group(type_code):
    squamous = [8051, 8052, 8070, 8071, 8072, 8074, 8075, 8082, 8083] #鱗狀細胞癌
    others = [8140, 8147, 8200, 8290, 8310, 8430, 8500, 8525.8550, 8560, 8562, 8982] #其他
    labels = ["鱗狀細胞癌", "其他"]
    hist = type_code//10

    if hist in squamous:
        return labels[0]
    if hist in others:
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

# 頰部 C06.0-C06.2  C06.8-C06.9
# 舌部 C02.0-C02.3 C02.8-C02.9
# 牙齦 C03.0-C03.1 C03.9
# 唇部 C00.0-C00.9
# 口底 C040-C041 C04.8-C04.9
# 硬顎 C05.0  C05.8-C05.9
def insite(site_org):
    Cheek = [60, 61, 62, 68, 69]
    Tongue  = [20, 21, 22, 23, 28, 29]
    Gingiva = [30, 31, 39]
    Lips = [i for i in range(0, 10)]
    Floor_of_mouth = [40, 41, 48, 49]
    Hard_palate = [50, 58, 5.9]

    if site_org in Cheek:
        return "頰部"
    if site_org in Tongue:
        return "舌部"
    if site_org in Gingiva:
        return "牙齦"
    if site_org in Lips:
        return "唇部"
    if site_org in Floor_of_mouth:
        return "口底"
    if site_org in Hard_palate:
        return "硬顎"
    else:
        return None


def oral_in_site_(df):
    df_site_distribution = df[['site-c', 'class']]
    df_site_distribution['class'] = df_site_distribution['class'].str.replace("'", '').astype(int)
    for i in range(len(df_site_distribution)):
        if int(df_site_distribution["class"][i]) not in range(0, 4):
            df_site_distribution = df_site_distribution.drop(df_site_distribution.columns[i], axis=1)

    df_site_distribution["site"] = None
    df_site_distribution["site"] = df_site_distribution["site-c"].apply(insite)
    site_distribution_ = df_site_distribution["site"].value_counts()
    site_distribution_ = pd.DataFrame(site_distribution_)

    #Save file
    site_distribution_.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_口腔癌原發部位.xlsx', sheet_name='site_class')
    return site_distribution_

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)
site_distribution_ = oral_in_site_(df)
order = ['1', '2', '3', '4A', '4B','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
OralCavity_therapy_ = therapy_order_Head_Neck_(df)

# OralCavity_therapy_

"""# carcinoma_type = "Oropharynx"
"""

carcinoma_type = "口咽癌"

def Type_Group(type_code):
    squamous16pos = [8070, 8072, 8083, 8085] #鱗狀細胞癌 P16+
    squamous16neg = [8051, 8052, 8070, 8074, 8075, 8082, 8083, 8086, 8560] #鱗狀細胞癌 P16-
    # others = []
    labels = ["鱗狀細胞癌 P16+", "鱗狀細胞癌 P16-"]
    hist = type_code//10

    if hist in squamous16pos:
        return labels[0]
    if hist in squamous16neg:
        return labels[1]
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌 P16+", "鱗狀細胞癌 P16-", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4A', '4B','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
Oropharynx_therapy_ = therapy_order_Head_Neck_(df)

# Oropharynx_therapy_

"""# carcinoma_type = "Hypopharynx"
"""

carcinoma_type = "下咽癌"

def Type_Group(type_code):
    squamous = [8051, 8052, 8070, 8074, 8075, 8082, 8083, 8086, 8560] #鱗狀細胞癌

    labels = ["鱗狀細胞癌"]
    hist = type_code//10

    if hist in squamous:
        return labels[0]
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4A', '4B','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
Hypopharynx_therapy_ = therapy_order_Head_Neck_(df)

# Hypopharynx_therapy_

"""# carcinoma_type = "Larynx"
"""

carcinoma_type = "喉癌"

def Type_Group(type_code):
    squamous = [8051, 8052, 8070, 8071, 8072, 8074, 8075, 8082, 8083, 8560] #鱗狀細胞癌
    others = [ 8013, 8041, 8045, 8200, 8240, 8249, 8430] #其他
    labels = ["鱗狀細胞癌", "其他"]
    hist = type_code//10

    if hist in squamous:
        return labels[0]
    if hist in others:
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4A', '4B','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
Larynx_therapy_ = therapy_order_Head_Neck_(df)

# Larynx_therapy_

"""# carcinoma_type = "Nasopharynx"
"""

carcinoma_type = "鼻咽癌"

def Type_Group(type_code):
    non_keratinizing  = [8072, 8073] #非角化鱗狀細胞癌
    _keratinizing  = [8071] #角化鱗狀細胞癌
    others = [8000, 8010, 8020, 8052, 8070, 8083, 8140, 8200] #其它
    labels = ["非角化鱗狀細胞癌", "角化鱗狀細胞癌"]
    hist = type_code//10

    if hist in non_keratinizing:
        return labels[0]
    if hist in _keratinizing:
        return labels[1]
    if hist in others:
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["非角化鱗狀細胞癌", "角化鱗狀細胞癌", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4A', '4B','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
Nasopharynx_therapy_ = therapy_order_Head_Neck_(df)

# Nasopharynx_therapy_

"""# carcinoma_type = "Esophagus"

### 食道癌治療順序
"""

def therapy_order_Esophagus(df):
    #### 資料選取
    therapy_order = ['class', 'optype_o', 'optype_h', 'rtmodal', 'ort_modal', 'srs', 'sls', 'chem_h', 'horm_h', 'immu_h', 'htep_h', 'targe_Tfacility', 'palli_h', 'other_T']
    therapy_df = df[therapy_order]
    for i in therapy_order:
        therapy_df[i] = therapy_df[i].str.replace(r"[^0-9]", '', regex=True)
        therapy_df[i] = pd.to_numeric(therapy_df[i], errors='coerce')
    therapy_df = therapy_df[therapy_df['class'].isin([1, 2])]
    therapy_df = therapy_df.reset_index()#.drop('index', axis=1)
    # therapy_df

    ########################################## "食道癌「治療」定義 " ##########################################
    # OP
    OP_df = therapy_df[~((therapy_df['optype_o'].isin([0, 99])) & (therapy_df['optype_h'].isin([0, 99])))]
    OP_df = OP_df.reset_index()#.drop('index', axis=1)
    OP_list = list(OP_df['index'])
    op = set(OP_list)
    # OP_list

    # RT
    RT_df = therapy_df[(therapy_df['rtmodal'] >= 1) & (therapy_df['rtmodal'] <= 64)]
    RT_df = RT_df.reset_index()#.drop('index', axis=1)
    RT_list = list(RT_df['index'])
    rt = set(RT_list)
    # RT_list


    # CHEM
    CHEM_df = therapy_df[(therapy_df['chem_h'] >= 1) & (therapy_df['chem_h'] <= 31)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem = set(CHEM_list)
    # CHEM_list

    # # HORM
    # HORM_df = therapy_df[(therapy_df['horm_h'] >= 1) & (therapy_df['horm_h'] <= 31)]
    # HORM_df = HORM_df.reset_index()#.drop('index', axis=1)
    # HORM_list = list(HORM_df['index'])
    # horm = set(HORM_list)
    # HORM_list

    # IMMU
    IMMU_df = therapy_df[(therapy_df['immu_h'] >= 1) & (therapy_df['immu_h'] <= 33)]
    IMMU_df = IMMU_df.reset_index()#.drop('index', axis=1)
    IMMU_list = list(IMMU_df['index'])
    immu = set(IMMU_list)
    # IMMU_list

    # # HTEP
    # HTEP_df = therapy_df[(therapy_df['htep_h'] >= 1) & (therapy_df['htep_h'] <= 31)]
    # HTEP_df = HTEP_df.reset_index()#.drop('index', axis=1)
    # HTEP_list = list(HTEP_df['index'])
    # htep = set(HTEP_list)
    # HTEP_list

    # TARGE
    # TARGE_df = therapy_df[(therapy_df['targe_Tfacility'] >= 1) & (therapy_df['targe_Tfacility'] <= 31)]
    # TARGE_df = TARGE_df.reset_index()#.drop('index', axis=1)
    # TARGE_list = list(TARGE_df['index'])
    # tg = set(TARGE_list)
    # TARGE_list

    # PALLI
    PALLI_df = therapy_df[(therapy_df['palli_h'] >= 1) & (therapy_df['palli_h'] <= 7)]
    PALLI_df = PALLI_df.reset_index()#.drop('index', axis=1)
    PALLI_list = list(PALLI_df['index'])
    palli_therapy = set(PALLI_list)
    # PALLI_list


    # Other
    Other_df = therapy_df[(therapy_df['other_T'] >= 1) & (therapy_df['other_T'] <= 3)]
    Other_df = Other_df.reset_index()#.drop('index', axis=1)
    Other_list = list(Other_df['index'])
    other_therapy = set(Other_list)
    # Other_list

    # 同步(~非同步~)化療與放療或/與標靶治療
    sls_df = therapy_df[(therapy_df['sls'].isin([2,6]))]
    sls_df = sls_df.reset_index()#.drop('index', axis=1)
    # sls_df
    sls_list = list(sls_df['index'])
    sls = set(sls_list)
    # sls_list
    ########################################## "食道癌「治療」定義 end" ##########################################

    ########################################## "食道癌「治療」範圍" ##########################################
    #### 狄摩根規則
    result_dict = {
        "治療方式": [],
        "人數": [],
        f"{carcinoma_type} 病患 Index List": []
    }


    # 手術
    OP_therapy = list(op - (rt | chem))
    # print(f"OP_therapy:\t\t\t\t\t\t{OP_therapy}")
    result_dict["治療方式"].append("手術")
    result_dict["人數"].append(len(OP_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_therapy if OP_therapy else None)


    # 手術合併化療與放療
    OP_RT_CHEM_combined_therapy = list(op & rt & chem)
    # print(f"OP_RT_CHEM_combined_therapy:\t\t\t\t{OP_RT_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療與放療")
    result_dict["人數"].append(len(OP_RT_CHEM_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_RT_CHEM_combined_therapy if OP_RT_CHEM_combined_therapy else None)


    # 手術合併放療
    OP_RT_combined_therapy = list(set(op & rt) - set(OP_RT_CHEM_combined_therapy))
    # print(f"OP_RT_combined_therapy:\t\t\t\t\t{OP_RT_combined_therapy}")
    result_dict["治療方式"].append("手術合併放療")
    result_dict["人數"].append(len(OP_RT_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_RT_combined_therapy if OP_RT_combined_therapy else None)


    # 手術合併化療
    OP_CHEM_combined_therapy = list(set(op & chem) - set(OP_RT_CHEM_combined_therapy))
    # print(f"OP_CHEM_combined_therapy:\t\t\t\t{OP_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療")
    result_dict["人數"].append(len(OP_CHEM_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_combined_therapy if OP_CHEM_combined_therapy else None)


    # （非）同步 化療與放療
    chem_rt_target_indexes = set(set(chem & rt) - set(OP_RT_CHEM_combined_therapy))
    # print(f"chem_rt_target_: {chem_rt_target_indexes}")
    sync_chem_rt_target_therapy = []   # 同步化療與放療或/與標靶治療 == sls
    async_chem_rt_target_therapy = []  # 非同步化療與放療或/與標靶治療
    for i in chem_rt_target_indexes:
        if i in sls:
            sync_chem_rt_target_therapy.append(i)
        else:
            async_chem_rt_target_therapy.append(i)
    # print(f"sync_chem_rt_target_therapy:\t\t\t\t{sync_chem_rt_target_therapy}\nasync_chem_rt_target_therapy:\t\t\t\t{async_chem_rt_target_therapy}")
    result_dict["治療方式"].append("同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(sync_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(sync_chem_rt_target_therapy if sync_chem_rt_target_therapy else None)


    result_dict["治療方式"].append("非同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(async_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(async_chem_rt_target_therapy if async_chem_rt_target_therapy else None)


    # 放療合併化療與免疫治療
    RT_CHEM_IMMU_combined_therapy = list(rt & chem & immu)
    # print(f"RT_CHEM_IMMU_combined_therapy:\t\t\t\t{RT_CHEM_IMMU_combined_therapy}")
    result_dict["治療方式"].append("放療合併化療與免疫治療")
    result_dict["人數"].append(len(RT_CHEM_IMMU_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(RT_CHEM_IMMU_combined_therapy if RT_CHEM_IMMU_combined_therapy else None)


    # 放療
    RT_therapy = list(rt - (op | chem))
    # print(f"RT_therapy:\t\t\t\t\t\t{RT_therapy}")
    result_dict["治療方式"].append("放療")
    result_dict["人數"].append(len(RT_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(RT_therapy if RT_therapy else None)


    # 化療合併免疫治療
    CHEM_IMMU_combined_therapy = list(set(immu & chem) - set(RT_CHEM_IMMU_combined_therapy))
    # print(f"CHEM_IMMU_combined_therapy:\t\t\t\t{CHEM_IMMU_combined_therapy}")
    result_dict["治療方式"].append("化療合併免疫治療")
    result_dict["人數"].append(len(CHEM_IMMU_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(CHEM_IMMU_combined_therapy if CHEM_IMMU_combined_therapy else None)


    # 化療
    chem_therapy = list(chem - (op | rt))
    # print(f"chem_therapy:\t\t\t\t\t\t{chem_therapy}")
    result_dict["治療方式"].append("化療或/與標靶治療")
    result_dict["人數"].append(len(chem_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(chem_therapy if chem_therapy else None)


    # 緩和治療
    palli_therapy = list(palli_therapy)
    # print(f"palli_therapy:\t\t\t\t\t\t{palli_therapy}")
    result_dict["治療方式"].append("緩和治療")
    result_dict["人數"].append(len(palli_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(palli_therapy if palli_therapy else None)


    # 其他治療
    other_therapy = list(other_therapy)
    # print(f"other_therapy:\t\t\t\t\t\t{other_therapy}")
    result_dict["治療方式"].append("其他治療")
    result_dict["人數"].append(len(other_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(other_therapy if other_therapy else None)

    # result_dict
    result_df = pd.DataFrame(result_dict)
    # result_df

    #Save file
    result_df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_therapy_type.xlsx', sheet_name='therapy')
    ########################################## "食道癌「治療」範圍 end" ##########################################
    return result_df

carcinoma_type = "食道癌"

def Type_Group(type_code):
    squamous  = [8051, 8070, 8071, 8074, 8077, 8083] #鱗狀細胞癌
    adenocarcinoma  = [8140, 8148, 8200, 8244, 8430] #腺癌
    adeno_squamous = [8560] #腺性鱗狀癌
    others = [8000, 8010, 8013, 8020, 8145, 8246, 8255] #其它
    labels = ["鱗狀細胞癌", "腺癌", "腺性鱗狀癌"]
    hist = type_code//10

    if hist in squamous:
        return labels[0]
    if hist in adenocarcinoma:
        return labels[1]
    if hist in adeno_squamous:
        return labels[2]
    if hist in others:
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌", "腺癌", "腺性鱗狀癌", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
Esophagus_therapy_ = therapy_order_Esophagus(df)

# Esophagus_therapy_

"""# carcinoma_type = "Stomach"

### 胃癌治療順序
"""

def therapy_order_Stomach(df):
    #### 資料選取
    therapy_order = ['class', 'optype_o', 'optype_h', 'rtmodal', 'ort_modal', 'srs', 'sls', 'chem_h', 'horm_h', 'immu_h', 'htep_h', 'targe_Tfacility', 'palli_h', 'other_T']
    therapy_df = df[therapy_order]
    for i in therapy_order:
        therapy_df[i] = therapy_df[i].str.replace(r"[^0-9]", '', regex=True)
        therapy_df[i] = pd.to_numeric(therapy_df[i], errors='coerce')
    therapy_df = therapy_df[therapy_df['class'].isin([1, 2])]
    therapy_df = therapy_df.reset_index()#.drop('index', axis=1)
    # therapy_df

    ########################################## "胃癌「治療」定義 " ##########################################
    # OP
    OP_df = therapy_df[~((therapy_df['optype_o'].isin([0, 99])) & (therapy_df['optype_h'].isin([0, 99])))]
    OP_df = OP_df.reset_index()#.drop('index', axis=1)
    OP_list = list(OP_df['index'])
    op = set(OP_list)
    # OP_list

    # RT
    RT_df = therapy_df[(therapy_df['rtmodal'] >= 1) & (therapy_df['rtmodal'] <= 64)]
    RT_df = RT_df.reset_index()#.drop('index', axis=1)
    RT_list = list(RT_df['index'])
    rt = set(RT_list)
    # RT_list


    # CHEM
    CHEM_df = therapy_df[(therapy_df['chem_h'] >= 1) & (therapy_df['chem_h'] <= 31) & (therapy_df['chem_h'] != 9)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem_general = set(CHEM_list)

    CHEM_df = therapy_df[(therapy_df['chem_h'] == 9)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem_local = set(CHEM_list)

    CHEM_df = therapy_df[(therapy_df['chem_h'] >= 1) & (therapy_df['chem_h'] <= 31)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem = set(CHEM_list)

    # CHEM_list


    # # HORM
    # HORM_df = therapy_df[(therapy_df['horm_h'] >= 1) & (therapy_df['horm_h'] <= 31)]
    # HORM_df = HORM_df.reset_index()#.drop('index', axis=1)
    # HORM_list = list(HORM_df['index'])
    # horm = set(HORM_list)
    # HORM_list

    # IMMU
    IMMU_df = therapy_df[(therapy_df['immu_h'] >= 1) & (therapy_df['immu_h'] <= 33)]
    IMMU_df = IMMU_df.reset_index()#.drop('index', axis=1)
    IMMU_list = list(IMMU_df['index'])
    immu = set(IMMU_list)
    # IMMU_list

    # # HTEP
    # HTEP_df = therapy_df[(therapy_df['htep_h'] >= 1) & (therapy_df['htep_h'] <= 31)]
    # HTEP_df = HTEP_df.reset_index()#.drop('index', axis=1)
    # HTEP_list = list(HTEP_df['index'])
    # htep = set(HTEP_list)
    # HTEP_list

    # TARGE
    TARGE_df = therapy_df[(therapy_df['targe_Tfacility'] >= 1) & (therapy_df['targe_Tfacility'] <= 31)]
    TARGE_df = TARGE_df.reset_index()#.drop('index', axis=1)
    TARGE_list = list(TARGE_df['index'])
    tg = set(TARGE_list)
    # TARGE_list

    # PALLI
    PALLI_df = therapy_df[(therapy_df['palli_h'] >= 1) & (therapy_df['palli_h'] <= 7)]
    PALLI_df = PALLI_df.reset_index()#.drop('index', axis=1)
    PALLI_list = list(PALLI_df['index'])
    palli_therapy = set(PALLI_list)
    # PALLI_list


    # Other
    Other_df = therapy_df[(therapy_df['other_T'] >= 1) & (therapy_df['other_T'] <= 3)]
    Other_df = Other_df.reset_index()#.drop('index', axis=1)
    Other_list = list(Other_df['index'])
    other_therapy = set(Other_list)
    # Other_list

    # 同步(~非同步~)化療與放療或/與標靶治療
    sls_df = therapy_df[(therapy_df['sls'].isin([2,6]))]
    sls_df = sls_df.reset_index()#.drop('index', axis=1)
    # sls_df
    sls_list = list(sls_df['index'])
    sls = set(sls_list)
    # sls_list
    ########################################## "胃癌「治療」定義 end" ##########################################

    ########################################## "胃癌「治療」範圍" ##########################################
    #### 狄摩根規則
    result_dict = {
        "治療方式": [],
        "人數": [],
        f"{carcinoma_type} 病患 Index List": []
    }


    # 手術
    OP_therapy = list(op - (rt | chem | tg))
    # print(f"OP_therapy:\t\t\t\t\t\t{OP_therapy}")
    result_dict["治療方式"].append("手術")
    result_dict["人數"].append(len(OP_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_therapy if OP_therapy else None)


    # 手術合併化療
    OP_CHEMgeneral_combined_therapy = list(set(op & chem_general) - set(op & chem & rt))
    # print(f"OP_CHEM_combined_therapy:\t\t\t\t{OP_CHEMgeneral_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療")
    result_dict["人數"].append(len(OP_CHEMgeneral_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEMgeneral_combined_therapy if OP_CHEMgeneral_combined_therapy else None)


    #手術合併全申身與局部化療
    OP_CHEMlocal_combined_therapy = list(set(op & chem_local) - set(op & chem  & rt))
    # print(f"OP_CHEM_combined_therapy:\t\t\t\t{OP_CHEMlocal_combined_therapy}")
    result_dict["治療方式"].append("手術合併全申身與局部化療")
    result_dict["人數"].append(len(OP_CHEMlocal_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEMlocal_combined_therapy if OP_CHEMlocal_combined_therapy else None)


    # 手術合併化療與放療
    OP_RT_CHEM_combined_therapy = list(op & rt & chem)
    # print(f"OP_RT_CHEM_combined_therapy:\t\t\t\t{OP_RT_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療與放療")
    result_dict["人數"].append(len(OP_RT_CHEM_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_RT_CHEM_combined_therapy if OP_RT_CHEM_combined_therapy else None)


    # 手術合併標靶治療
    OP_TG_combined_therapy = list(set(op & tg))
    # print(f"OP_RT_combined_therapy:\t\t\t\t\t{OP_TG_combined_therapy}")
    result_dict["治療方式"].append("手術合併標靶治療")
    result_dict["人數"].append(len(OP_TG_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_TG_combined_therapy if OP_TG_combined_therapy else None)


    #手術合併化療與免疫治療
    OP_CHEM_IMMU_combined_therapy = list(op & immu & chem)
    # print(f"OP_RT_CHEM_combined_therapy:\t\t\t\t{OP_CHEM_IMMU_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療與免疫治療")
    result_dict["人數"].append(len(OP_CHEM_IMMU_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_IMMU_combined_therapy if OP_CHEM_IMMU_combined_therapy else None)


    # 化療
    chem_therapy = list(chem - (op | rt | immu))
    # print(f"chem_therapy:\t\t\t\t\t\t{chem_therapy}")
    result_dict["治療方式"].append("化療或/與標靶治療")
    result_dict["人數"].append(len(chem_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(chem_therapy if chem_therapy else None)


    # 化療合併免疫治療
    CHEM_IMMU_combined_therapy = list(set(immu & chem) - set(op & immu & chem))
    # print(f"CHEM_IMMU_combined_therapy:\t\t\t\t{CHEM_IMMU_combined_therapy}")
    result_dict["治療方式"].append("化療合併免疫治療")
    result_dict["人數"].append(len(CHEM_IMMU_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(CHEM_IMMU_combined_therapy if CHEM_IMMU_combined_therapy else None)


    # 放療
    RT_therapy = list(rt - (op | chem))
    # print(f"RT_therapy:\t\t\t\t\t\t{RT_therapy}")
    result_dict["治療方式"].append("放療")
    result_dict["人數"].append(len(RT_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(RT_therapy if RT_therapy else None)


    # （非）同步 化療與放療
    chem_rt_target_indexes = set(set(chem & rt) - set(op & rt & chem))
    # print(f"chem_rt_target_: {chem_rt_target_indexes}")
    sync_chem_rt_target_therapy = []   # 同步化療與放療或/與標靶治療 == sls
    async_chem_rt_target_therapy = []  # 非同步化療與放療或/與標靶治療
    for i in chem_rt_target_indexes:
        if i in sls:
            sync_chem_rt_target_therapy.append(i)
        else:
            async_chem_rt_target_therapy.append(i)
    # print(f"sync_chem_rt_target_therapy:\t\t\t\t{sync_chem_rt_target_therapy}\nasync_chem_rt_target_therapy:\t\t\t\t{async_chem_rt_target_therapy}")
    result_dict["治療方式"].append("同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(sync_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(sync_chem_rt_target_therapy if sync_chem_rt_target_therapy else None)
    result_dict["治療方式"].append("非同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(async_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(async_chem_rt_target_therapy if async_chem_rt_target_therapy else None)


    #標靶治療
    tg_therapy = list(tg - (tg & op))
    # print(f"chem_therapy:\t\t\t\t\t\t{tg_therapy}")
    result_dict["治療方式"].append("標靶治療")
    result_dict["人數"].append(len(tg_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(tg_therapy if tg_therapy else None)


    # 緩和治療
    palli_therapy = list(palli_therapy)
    # print(f"palli_therapy:\t\t\t\t\t\t{palli_therapy}")
    result_dict["治療方式"].append("緩和治療")
    result_dict["人數"].append(len(palli_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(palli_therapy if palli_therapy else None)


    # 其他治療
    other_therapy = list(other_therapy)
    # print(f"other_therapy:\t\t\t\t\t\t{other_therapy}")
    result_dict["治療方式"].append("其他治療")
    result_dict["人數"].append(len(other_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(other_therapy if other_therapy else None)

    # result_dict
    result_df = pd.DataFrame(result_dict)
    # result_df

    #Save file
    result_df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_therapy_type.xlsx', sheet_name='therapy')
    ########################################## "胃癌「治療」範圍 end" ##########################################
    return result_df

carcinoma_type = "胃癌"

def Type_Group(type_code):
    squamous  = [8070, 8082] #鱗狀細胞癌
    adenocarcinoma  = [8020, 8140, 8142, 8144, 8145, 8148, 8211, 8214, 8244, 8260, 8480, 8490, 8510, 8576] #腺癌
    adeno_squamous = [8560] #腺性鱗狀癌
    coma = [8936] #胃腸基質瘤
    others = [8000, 8010, 8013, 8041, 8246, 8255] #其它
    labels = ["鱗狀細胞癌", "腺癌", "腺性鱗狀癌", "胃腸基質瘤"]
    hist = type_code//10

    if hist in squamous:
        return labels[0]
    if hist in adenocarcinoma:
        return labels[1]
    if hist in adeno_squamous:
        return labels[2]
    if hist in coma:
        return labels[3]
    if hist in others:
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌", "腺癌", "腺性鱗狀癌", "胃腸基質瘤", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
Stomach_therapy_ = therapy_order_Stomach(df)

Stomach_therapy_

"""# carcinoma_type = "Colon"

### 結腸癌治療順序
"""

def therapy_order_Colon_(df):
    #### 資料選取
    therapy_order = ['class', 'optype_o', 'optype_h', 'rtmodal', 'ort_modal', 'srs', 'sls', 'chem_h', 'horm_h', 'immu_h', 'htep_h', 'targe_Tfacility', 'palli_h', 'other_T']
    therapy_df = df[therapy_order]
    for i in therapy_order:
        therapy_df[i] = therapy_df[i].str.replace(r"[^0-9]", '', regex=True)
        therapy_df[i] = pd.to_numeric(therapy_df[i], errors='coerce')
    therapy_df = therapy_df[therapy_df['class'].isin([1, 2])]
    therapy_df = therapy_df.reset_index()#.drop('index', axis=1)
    # therapy_df

    ########################################## "結腸癌「治療」定義 " ##########################################
    # OP
    OP_df = therapy_df[~((therapy_df['optype_o'].isin([0, 99])) & (therapy_df['optype_h'].isin([0, 99])))]
    OP_df = OP_df.reset_index()#.drop('index', axis=1)
    OP_list = list(OP_df['index'])
    op = set(OP_list)
    # OP_list

    # RT
    RT_df = therapy_df[(therapy_df['rtmodal'] >= 1) & (therapy_df['rtmodal'] <= 64)]
    RT_df = RT_df.reset_index()#.drop('index', axis=1)
    RT_list = list(RT_df['index'])
    rt = set(RT_list)
    # RT_list


    # CHEM
    CHEM_df = therapy_df[(therapy_df['chem_h'] >= 1) & (therapy_df['chem_h'] <= 31)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem = set(CHEM_list)
    # CHEM_list

    # # HORM
    # HORM_df = therapy_df[(therapy_df['horm_h'] >= 1) & (therapy_df['horm_h'] <= 31)]
    # HORM_df = HORM_df.reset_index()#.drop('index', axis=1)
    # HORM_list = list(HORM_df['index'])
    # horm = set(HORM_list)
    # HORM_list

    # # IMMU
    # IMMU_df = therapy_df[(therapy_df['immu_h'] >= 10) & (therapy_df['immu_h'] <= 50)]
    # IMMU_df = IMMU_df.reset_index()#.drop('index', axis=1)
    # IMMU_list = list(IMMU_df['index'])
    # immu = set(IMMU_list)
    # IMMU_list

    # # HTEP
    # HTEP_df = therapy_df[(therapy_df['htep_h'] >= 1) & (therapy_df['htep_h'] <= 31)]
    # HTEP_df = HTEP_df.reset_index()#.drop('index', axis=1)
    # HTEP_list = list(HTEP_df['index'])
    # htep = set(HTEP_list)
    # HTEP_list

    # TARGE
    TARGE_df = therapy_df[(therapy_df['targe_Tfacility'] >= 1) & (therapy_df['targe_Tfacility'] <= 31)]
    TARGE_df = TARGE_df.reset_index()#.drop('index', axis=1)
    TARGE_list = list(TARGE_df['index'])
    tg = set(TARGE_list)
    # TARGE_list

    # PALLI
    PALLI_df = therapy_df[(therapy_df['palli_h'] >= 1) & (therapy_df['palli_h'] <= 7)]
    PALLI_df = PALLI_df.reset_index()#.drop('index', axis=1)
    PALLI_list = list(PALLI_df['index'])
    palli_therapy = set(PALLI_list)
    # PALLI_list


    # Other
    Other_df = therapy_df[(therapy_df['other_T'] >= 1) & (therapy_df['other_T'] <= 3)]
    Other_df = Other_df.reset_index()#.drop('index', axis=1)
    Other_list = list(Other_df['index'])
    other_therapy = set(Other_list)
    # Other_list

    # 同步(~非同步~)化療與放療或/與標靶治療
    sls_df = therapy_df[(therapy_df['sls'].isin([2,6]))]
    sls_df = sls_df.reset_index()#.drop('index', axis=1)
    # sls_df
    sls_list = list(sls_df['index'])
    sls = set(sls_list)
    # sls_list
    ########################################## "結腸癌「治療」定義 end" ##########################################

    ########################################## "結腸癌「治療」範圍" ##########################################
    #### 狄摩根規則
    result_dict = {
        "治療方式": [],
        "人數": [],
        f"{carcinoma_type} 病患 Index List": []
    }


    # 手術
    OP_therapy = list(op - (rt | chem | tg))
    # print(f"OP_therapy:\t\t\t\t\t\t{OP_therapy}")
    result_dict["治療方式"].append("手術")
    result_dict["人數"].append(len(OP_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_therapy if OP_therapy else None)


    # 手術合併化療、放療與標靶治療
    OP_CHEM_RT_TG_combined_therapy = list(op & rt & chem & tg)
    # print(f"OP_RT_CHEM_combined_therapy:\t\t\t\t{OP_CHEM_RT_TG_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療、放療與標靶治療")
    result_dict["人數"].append(len(OP_CHEM_RT_TG_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_RT_TG_combined_therapy if OP_CHEM_RT_TG_combined_therapy else None)


    # 手術合併化療與標靶治療
    OP_CHEM_TG_combined_therapy = list(set(op & chem & tg) - set(op & rt & chem & tg))
    # print(f"OP_RT_CHEM_combined_therapy:\t\t\t\t{OP_CHEM_TG_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療與標靶治療")
    result_dict["人數"].append(len(OP_CHEM_TG_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_TG_combined_therapy if OP_CHEM_TG_combined_therapy else None)


    # 手術合併化療與放療
    OP_CHEM_RT_combined_therapy = list(set(op & rt & chem) - set(op & rt & chem & tg))
    # print(f"OP_RT_CHEM_combined_therapy:\t\t\t\t{OP_RT_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療與放療")
    result_dict["人數"].append(len(OP_CHEM_RT_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_RT_combined_therapy if OP_CHEM_RT_combined_therapy else None)


#     # 手術合併放療
#     OP_RT_combined_therapy = list(set(op & rt) - set(OP_RT_CHEM_combined_therapy))
#     # print(f"OP_RT_combined_therapy:\t\t\t\t\t{OP_RT_combined_therapy}")
#     result_dict["治療方式"].append("手術合併放療")
#     result_dict["人數"].append(len(OP_RT_combined_therapy))
#     result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_RT_combined_therapy if OP_RT_combined_therapy else None)


    # 手術合併化療
    OP_CHEM_combined_therapy = list(set(op & chem) - set(op & chem & tg) - set(op & chem & rt))
    # print(f"OP_CHEM_combined_therapy:\t\t\t\t{OP_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療")
    result_dict["人數"].append(len(OP_CHEM_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_combined_therapy if OP_CHEM_combined_therapy else None)


    # 非同步 化療與放療或/與標靶治療
    chem_rt_target_indexes = set(set(chem & rt) - set(op & rt & chem))
    # print(f"chem_rt_target_: {chem_rt_target_indexes}")
    sync_chem_rt_target_therapy = []   # 同步化療與放療或/與標靶治療 == sls
    async_chem_rt_target_therapy = []  # 非同步化療與放療或/與標靶治療
    for i in chem_rt_target_indexes:
        if i in sls:
            sync_chem_rt_target_therapy.append(i)
        else:
            async_chem_rt_target_therapy.append(i)
    # print(f"sync_chem_rt_target_therapy:\t\t\t\t{sync_chem_rt_target_therapy}\nasync_chem_rt_target_therapy:\t\t\t\t{async_chem_rt_target_therapy}")
#     result_dict["治療方式"].append("同步化療與放療或/與標靶治療")
#     result_dict["人數"].append(len(sync_chem_rt_target_therapy))
#     result_dict[f"{carcinoma_type} 病患 Index List"].append(sync_chem_rt_target_therapy if sync_chem_rt_target_therapy else None)
    result_dict["治療方式"].append("非同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(async_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(async_chem_rt_target_therapy if async_chem_rt_target_therapy else None)


#     # 放療
#     RT_therapy = list(rt - (op | chem))
#     # print(f"RT_therapy:\t\t\t\t\t\t{RT_therapy}")
#     result_dict["治療方式"].append("放療")
#     result_dict["人數"].append(len(RT_therapy))
#     result_dict[f"{carcinoma_type} 病患 Index List"].append(RT_therapy if RT_therapy else None)


    # 化療或/與標靶治療
    chem_therapy = list(chem - (op | rt | tg))
    # print(f"chem_therapy:\t\t\t\t\t\t{chem_therapy}")
    result_dict["治療方式"].append("化療或/與標靶治療")
    result_dict["人數"].append(len(chem_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(chem_therapy if chem_therapy else None)


    # 緩和治療
    palli_therapy = list(palli_therapy)
    # print(f"palli_therapy:\t\t\t\t\t\t{palli_therapy}")
    result_dict["治療方式"].append("緩和治療")
    result_dict["人數"].append(len(palli_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(palli_therapy if palli_therapy else None)


    # 其他治療
    other_therapy = list(other_therapy)
    # print(f"other_therapy:\t\t\t\t\t\t{other_therapy}")
    result_dict["治療方式"].append("其他治療")
    result_dict["人數"].append(len(other_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(other_therapy if other_therapy else None)

    # result_dict
    result_df = pd.DataFrame(result_dict)
    # result_df

    #Save file
    result_df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_therapy_type.xlsx', sheet_name='therapy')
    ########################################## "結腸癌「治療」範圍 end" ##########################################
    return result_df

carcinoma_type = "結腸癌"

def Type_Group(type_code):
    squamous  = [8051, 8070, 8071, 8072, 8077, 8083, 8090] #鱗狀細胞癌
    adenocarcinoma  = [8140, 8213, 8244, 8265, 8480, 8481, 8490, 8510] #腺癌
    stomach_coma = [8936] #胃腸基質瘤
    nurv_coma = [8240, 8241, 8246, 8249] #神經內分泌瘤
    others = [8000, 8010, 8013, 8020, 8041, 8123, 8124, 8542, 8560] #其它
    labels = ["鱗狀細胞癌", "腺癌", "胃腸基質瘤", "神經內分泌瘤"]
    hist = type_code//10

    if hist in squamous:
        return labels[0]
    if hist in adenocarcinoma:
        return labels[1]
    if hist in stomach_coma:
        return labels[2]
    if hist in nurv_coma:
        return labels[3]
    if hist in others:
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌", "腺癌", "腺性鱗狀癌", "神經內分泌瘤", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
Colon_therapy_ = therapy_order_Colon_(df)

Colon_therapy_

"""# carcinoma_type = "Rectum"

### 直腸癌治療順序
"""

def therapy_order_Rectum_(df):
    #### 資料選取
    therapy_order = ['class', 'optype_o', 'optype_h', 'rtmodal', 'ort_modal', 'srs', 'sls', 'chem_h', 'horm_h', 'immu_h', 'htep_h', 'targe_Tfacility', 'palli_h', 'other_T']
    therapy_df = df[therapy_order]
    for i in therapy_order:
        therapy_df[i] = therapy_df[i].str.replace(r"[^0-9]", '', regex=True)
        therapy_df[i] = pd.to_numeric(therapy_df[i], errors='coerce')
    therapy_df = therapy_df[therapy_df['class'].isin([1, 2])]
    therapy_df = therapy_df.reset_index()#.drop('index', axis=1)
    # therapy_df

    ########################################## "直腸癌「治療」定義 " ##########################################
    # OP
    OP_df = therapy_df[~((therapy_df['optype_o'].isin([0, 99])) & (therapy_df['optype_h'].isin([0, 99])))]
    OP_df = OP_df.reset_index()#.drop('index', axis=1)
    OP_list = list(OP_df['index'])
    op = set(OP_list)
    # OP_list

    # RT
    RT_df = therapy_df[(therapy_df['rtmodal'] >= 1) & (therapy_df['rtmodal'] <= 64)]
    RT_df = RT_df.reset_index()#.drop('index', axis=1)
    RT_list = list(RT_df['index'])
    rt = set(RT_list)
    # RT_list


    # CHEM
    CHEM_df = therapy_df[(therapy_df['chem_h'] >= 1) & (therapy_df['chem_h'] <= 31)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem = set(CHEM_list)
    # CHEM_list

    # # HORM
    # HORM_df = therapy_df[(therapy_df['horm_h'] >= 1) & (therapy_df['horm_h'] <= 31)]
    # HORM_df = HORM_df.reset_index()#.drop('index', axis=1)
    # HORM_list = list(HORM_df['index'])
    # horm = set(HORM_list)
    # HORM_list

    # # IMMU
    # IMMU_df = therapy_df[(therapy_df['immu_h'] >= 10) & (therapy_df['immu_h'] <= 50)]
    # IMMU_df = IMMU_df.reset_index()#.drop('index', axis=1)
    # IMMU_list = list(IMMU_df['index'])
    # immu = set(IMMU_list)
    # IMMU_list

    # # HTEP
    # HTEP_df = therapy_df[(therapy_df['htep_h'] >= 1) & (therapy_df['htep_h'] <= 31)]
    # HTEP_df = HTEP_df.reset_index()#.drop('index', axis=1)
    # HTEP_list = list(HTEP_df['index'])
    # htep = set(HTEP_list)
    # HTEP_list

    # TARGE
    TARGE_df = therapy_df[(therapy_df['targe_Tfacility'] >= 1) & (therapy_df['targe_Tfacility'] <= 31)]
    TARGE_df = TARGE_df.reset_index()#.drop('index', axis=1)
    TARGE_list = list(TARGE_df['index'])
    tg = set(TARGE_list)
    # TARGE_list

    # PALLI
    PALLI_df = therapy_df[(therapy_df['palli_h'] >= 1) & (therapy_df['palli_h'] <= 7)]
    PALLI_df = PALLI_df.reset_index()#.drop('index', axis=1)
    PALLI_list = list(PALLI_df['index'])
    palli_therapy = set(PALLI_list)
    # PALLI_list


    # Other
    Other_df = therapy_df[(therapy_df['other_T'] >= 1) & (therapy_df['other_T'] <= 3)]
    Other_df = Other_df.reset_index()#.drop('index', axis=1)
    Other_list = list(Other_df['index'])
    other_therapy = set(Other_list)
    # Other_list

    # 同步(~非同步~)化療與放療或/與標靶治療
    sls_df = therapy_df[(therapy_df['sls'].isin([2,6]))]
    sls_df = sls_df.reset_index()#.drop('index', axis=1)
    # sls_df
    sls_list = list(sls_df['index'])
    sls = set(sls_list)
    # sls_list
    ########################################## "直腸癌「治療」定義 end" ##########################################

    ########################################## "直腸癌「治療」範圍" ##########################################
    #### 狄摩根規則
    result_dict = {
        "治療方式": [],
        "人數": [],
        f"{carcinoma_type} 病患 Index List": []
    }


    # 手術
    OP_therapy = list(op - (rt | chem | tg))
    # print(f"OP_therapy:\t\t\t\t\t\t{OP_therapy}")
    result_dict["治療方式"].append("手術")
    result_dict["人數"].append(len(OP_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_therapy if OP_therapy else None)


    # 手術合併化療與放療
    OP_CHEM_RT_combined_therapy = list(set(op & rt & chem) - set(op & rt & chem & tg))
    # print(f"OP_RT_CHEM_combined_therapy:\t\t\t\t{OP_RT_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療與放療")
    result_dict["人數"].append(len(OP_CHEM_RT_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_RT_combined_therapy if OP_CHEM_RT_combined_therapy else None)


    # 手術合併放療
    OP_RT_combined_therapy = list(set(op & rt) - set(op & rt & chem))
    # print(f"OP_RT_combined_therapy:\t\t\t\t\t{OP_RT_combined_therapy}")
    result_dict["治療方式"].append("手術合併放療")
    result_dict["人數"].append(len(OP_RT_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_RT_combined_therapy if OP_RT_combined_therapy else None)


    # 手術合併化療
    OP_CHEM_combined_therapy = list(set(op & chem) - set(op & chem & rt))
    # print(f"OP_CHEM_combined_therapy:\t\t\t\t{OP_CHEM_combined_therapy}")
    result_dict["治療方式"].append("手術合併化療")
    result_dict["人數"].append(len(OP_CHEM_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_combined_therapy if OP_CHEM_combined_therapy else None)


    # （非）同步 化療與放療或/與標靶治療
    chem_rt_target_indexes = set(set(chem & rt) - set(op & rt & chem))
    # print(f"chem_rt_target_: {chem_rt_target_indexes}")
    sync_chem_rt_target_therapy = []   # 同步化療與放療或/與標靶治療 == sls
    async_chem_rt_target_therapy = []  # 非同步化療與放療或/與標靶治療
    for i in chem_rt_target_indexes:
        if i in sls:
            sync_chem_rt_target_therapy.append(i)
        else:
            async_chem_rt_target_therapy.append(i)
    # print(f"sync_chem_rt_target_therapy:\t\t\t\t{sync_chem_rt_target_therapy}\nasync_chem_rt_target_therapy:\t\t\t\t{async_chem_rt_target_therapy}")
    result_dict["治療方式"].append("同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(sync_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(sync_chem_rt_target_therapy if sync_chem_rt_target_therapy else None)
    result_dict["治療方式"].append("非同步化療與放療或/與標靶治療")
    result_dict["人數"].append(len(async_chem_rt_target_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(async_chem_rt_target_therapy if async_chem_rt_target_therapy else None)


    # 放療
    RT_therapy = list(rt - (op | chem))
    # print(f"RT_therapy:\t\t\t\t\t\t{RT_therapy}")
    result_dict["治療方式"].append("放療")
    result_dict["人數"].append(len(RT_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(RT_therapy if RT_therapy else None)


    # 化療或/與標靶治療
    chem_therapy = list(chem - (op | rt | tg))
    # print(f"chem_therapy:\t\t\t\t\t\t{chem_therapy}")
    result_dict["治療方式"].append("化療或/與標靶治療")
    result_dict["人數"].append(len(chem_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(chem_therapy if chem_therapy else None)


    # 緩和治療
    palli_therapy = list(palli_therapy)
    # print(f"palli_therapy:\t\t\t\t\t\t{palli_therapy}")
    result_dict["治療方式"].append("緩和治療")
    result_dict["人數"].append(len(palli_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(palli_therapy if palli_therapy else None)


    # 其他治療
    other_therapy = list(other_therapy)
    # print(f"other_therapy:\t\t\t\t\t\t{other_therapy}")
    result_dict["治療方式"].append("其他治療")
    result_dict["人數"].append(len(other_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(other_therapy if other_therapy else None)

    # result_dict
    result_df = pd.DataFrame(result_dict)
    # result_df

    #Save file
    result_df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_therapy_type.xlsx', sheet_name='therapy')
    ########################################## "直腸癌「治療」範圍 end" ##########################################
    return result_df

carcinoma_type = "直腸癌"

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)
Rectum_therapy_ = therapy_order_Rectum_(df)

"""# carcinoma_type = "Anal"
"""

carcinoma_type = "肛門癌"

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)

"""# carcinoma_type = "Liver"

### 肝癌治療順序
"""

def therapy_order_Liver_(df):
    #### 資料選取
    therapy_order = ['class', 'optype_o', 'optype_h', 'rtmodal', 'ort_modal', 'srs', 'sls', 'chem_h', 'horm_h', 'immu_h', 'htep_h', 'targe_Tfacility', 'palli_h', 'other_T']
    therapy_df = df[therapy_order]
    for i in therapy_order:
        therapy_df[i] = therapy_df[i].str.replace(r"[^0-9]", '', regex=True)
        therapy_df[i] = pd.to_numeric(therapy_df[i], errors='coerce')
    therapy_df = therapy_df[therapy_df['class'].isin([1, 2])]
    therapy_df = therapy_df.reset_index()#.drop('index', axis=1)
    # therapy_df

    ########################################## "肝癌「治療」定義 " ##########################################
    # OP\Transplant(tp)\Transcatheter(tc)
    OP_df = therapy_df[~((therapy_df['optype_o'].isin([0, 99])) & (therapy_df['optype_h'].isin([0, 99]))) & (~therapy_df['optype_o'].isin([61, 75, 15]))]
    OP_df = OP_df.reset_index()#.drop('index', axis=1)
    OP_list = list(OP_df['index'])
    op_general = set(OP_list)
    transplant_df = therapy_df[(therapy_df['optype_o'].isin([61, 75]))]
    transplant_df = transplant_df.reset_index()
    transplant_list = list(transplant_df['index'])
    tp = set(transplant_list)
    transcatheter_df = therapy_df[(therapy_df['optype_o'].isin([15]))]
    transcatheter_df = transcatheter_df.reset_index()
    transcatheter_list = list(transcatheter_df['index'])
    tc = set(transcatheter_list)
    # OP_list
    # transplant_list
    # transcatheter_list

    # RT
    RT_df = therapy_df[(therapy_df['rtmodal'] >= 1) & (therapy_df['rtmodal'] <= 64)]
    RT_df = RT_df.reset_index()#.drop('index', axis=1)
    RT_list = list(RT_df['index'])
    rt = set(RT_list)
    # RT_list


    # CHEM
    CHEM_df = therapy_df[(therapy_df['chem_h'] >= 4) & (therapy_df['chem_h'] <= 7)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem_local = set(CHEM_list)
    CHEM_df = therapy_df[(therapy_df['chem_h'] >= 0) & (therapy_df['chem_h'] <= 31)]
    CHEM_df = CHEM_df.reset_index()#.drop('index', axis=1)
    CHEM_list = list(CHEM_df['index'])
    chem_general = set(CHEM_list)
    # CHEM_list

    # # HORM
    # HORM_df = therapy_df[(therapy_df['horm_h'] >= 1) & (therapy_df['horm_h'] <= 31)]
    # HORM_df = HORM_df.reset_index()#.drop('index', axis=1)
    # HORM_list = list(HORM_df['index'])
    # horm = set(HORM_list)
    # HORM_list

    # # IMMU
    # IMMU_df = therapy_df[(therapy_df['immu_h'] >= 10) & (therapy_df['immu_h'] <= 50)]
    # IMMU_df = IMMU_df.reset_index()#.drop('index', axis=1)
    # IMMU_list = list(IMMU_df['index'])
    # immu = set(IMMU_list)
    # IMMU_list

    # # HTEP
    # HTEP_df = therapy_df[(therapy_df['htep_h'] >= 1) & (therapy_df['htep_h'] <= 31)]
    # HTEP_df = HTEP_df.reset_index()#.drop('index', axis=1)
    # HTEP_list = list(HTEP_df['index'])
    # htep = set(HTEP_list)
    # HTEP_list

    # TARGE
    TARGE_df = therapy_df[(therapy_df['targe_Tfacility'] >= 1) & (therapy_df['targe_Tfacility'] <= 31)]
    TARGE_df = TARGE_df.reset_index()#.drop('index', axis=1)
    TARGE_list = list(TARGE_df['index'])
    tg = set(TARGE_list)
    # TARGE_list

    # PALLI
    PALLI_df = therapy_df[(therapy_df['palli_h'] >= 1) & (therapy_df['palli_h'] <= 7)]
    PALLI_df = PALLI_df.reset_index()#.drop('index', axis=1)
    PALLI_list = list(PALLI_df['index'])
    palli_therapy = set(PALLI_list)
    # PALLI_list


    # Other
    Other_df = therapy_df[(therapy_df['other_T'] >= 1) & (therapy_df['other_T'] <= 3)]
    Other_df = Other_df.reset_index()#.drop('index', axis=1)
    Other_list = list(Other_df['index'])
    other_therapy = set(Other_list)
    # Other_list

    # 同步(~非同步~)化療與放療或/與標靶治療
    sls_df = therapy_df[(therapy_df['sls'].isin([2,6]))]
    sls_df = sls_df.reset_index()#.drop('index', axis=1)
    # sls_df
    sls_list = list(sls_df['index'])
    sls = set(sls_list)
    # sls_list
    ########################################## "肝癌「治療」定義 end" ##########################################

    ########################################## "肝癌「治療」範圍" ##########################################
    #### 狄摩根規則
    result_dict = {
        "治療方式": [],
        "人數": [],
        f"{carcinoma_type} 病患 Index List": []
    }


    # 手術
    OP_therapy = list(op_general - (chem_local | tc | tg))
    result_dict["治療方式"].append("手術")
    result_dict["人數"].append(len(OP_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_therapy if OP_therapy else None)

    # 肝移植
    TP_therapy = list(tp)
    result_dict["治療方式"].append("肝移植")
    result_dict["人數"].append(len(TP_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(TP_therapy if TP_therapy else None)

    # 局部治療
    chemlocal_therapy = list(chem_local - (op | tc))
    result_dict["治療方式"].append("局部治療")
    result_dict["人數"].append(len(chemlocal_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(chemlocal_therapy if chemlocal_therapy else None)

    # 栓塞
    TC_therapy = list(tc - (op | chem_local | tg | rt))
    result_dict["治療方式"].append("栓塞")
    result_dict["人數"].append(len(TC_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(TC_therapy if TC_therapy else None)


    #手術合併局部治療
    OP_CHEM_local_combined_therapy = list(set(op & chem_local))
    result_dict["治療方式"].append("手術合併局部治療")
    result_dict["人數"].append(len(OP_CHEM_local_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_CHEM_local_combined_therapy if OP_CHEM_local_combined_therapy else None)

    #手術合併栓塞
    OP_TC_combined_therapy = list(set(op & tc))
    result_dict["治療方式"].append("手術合併栓塞")
    result_dict["人數"].append(len(OP_TC_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_TC_combined_therapy if OP_TC_combined_therapy else None)

    #手術合併標靶治療
    OP_TP_combined_therapy = list(set(op & tp))
    result_dict["治療方式"].append("手術合併標靶治療")
    result_dict["人數"].append(len(OP_TP_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(OP_TP_combined_therapy if OP_TP_combined_therapy else None)


    # 局部治療合併栓塞
    TC_CHEM_local_combined_therapy = list(set(tc & chem_local))
    result_dict["治療方式"].append("局部治療合併栓塞")
    result_dict["人數"].append(len(TC_CHEM_local_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(TC_CHEM_local_combined_therapy if TC_CHEM_local_combined_therapy else None)

    # 標靶治療
    TG_therapy = list(tg - (op | tc | rt | chem_general | immu))
    result_dict["治療方式"].append("標靶治療")
    result_dict["人數"].append(len(tg_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(tg_therapy if tg_therapy else None)


    # 栓塞合併標靶治療
    TC_TG_combined_therapy = list(set(tc & tg) - set(rt & tc & tg))
    result_dict["治療方式"].append("栓塞合併標靶治療")
    result_dict["人數"].append(len(TC_TG_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(TC_TG_combined_therapy if TC_TG_combined_therapy else None)

    # 放療合併栓塞
    TC_RT_combined_therapy = list(set(tc & rt) - set(rt & tc & tg))
    result_dict["治療方式"].append("放療合併栓塞")
    result_dict["人數"].append(len(TC_RT_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(TC_RT_combined_therapy if TC_RT_combined_therapy else None)


    # 化療
    CHEM_therapy = list(chem_general - tg)
    result_dict["治療方式"].append("化療")
    result_dict["人數"].append(len(CHEM_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(CHEM_therapy if CHEM_therapy else None)


    # 化療合併標靶治療
    CHEM_TG_combined_therapy = list(set(chem_general & tg))
    result_dict["治療方式"].append("化療合併標靶治療")
    result_dict["人數"].append(len(CHEM_TG_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(CHEM_TG_combined_therapy if CHEM_TG_combined_therapy else None)


    # 免疫治療
    IMMU_therapy = list(immu - tg)
    result_dict["治療方式"].append("免疫治療")
    result_dict["人數"].append(len(IMMU_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(IMMU_therapy if IMMU_therapy else None)


    # 化療合併標靶治療
    IMMU_TG_combined_therapy = list(set(immu & tg))
    result_dict["治療方式"].append("化療合併標靶治療")
    result_dict["人數"].append(len(IMMU_TG_combined_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(IMMU_TG_combined_therapy if IMMU_TG_combined_therapy else None)



    # 緩和治療
    palli_therapy = list(palli_therapy)
    # print(f"palli_therapy:\t\t\t\t\t\t{palli_therapy}")
    result_dict["治療方式"].append("緩和治療")
    result_dict["人數"].append(len(palli_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(palli_therapy if palli_therapy else None)


    # 其他治療
    other_therapy = list(other_therapy)
    # print(f"other_therapy:\t\t\t\t\t\t{other_therapy}")
    result_dict["治療方式"].append("其他治療")
    result_dict["人數"].append(len(other_therapy))
    result_dict[f"{carcinoma_type} 病患 Index List"].append(other_therapy if other_therapy else None)

    # result_dict
    result_df = pd.DataFrame(result_dict)
    # result_df

    #Save file
    result_df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_therapy_type.xlsx', sheet_name='therapy')
    ########################################## "肝癌「治療」範圍 end" ##########################################
    return result_df

carcinoma_type = "肝癌"

def Type_Group(type_code):
    hepatocellular  = [8170, 8171, 8172, 8173, 8174] #肝細胞癌
    cholangiocarcinoma  = [8160, 8161] #膽管癌
    hepatocellular_cholangiocarcinoma = [8180] #合併肝細胞及膽管癌
    others = [8013, 8041, 8148, 8470, 8503, 8980] #其它
    labels = ["肝細胞癌", "膽管癌", "合併肝細胞及膽管癌"]
    hist = type_code//10

    if hist in hepatocellular:
        return labels[0]
    if hist in cholangiocarcinoma:
        return labels[1]
    if hist in hepatocellular_cholangiocarcinoma:
        return labels[2]
    if hist in others:
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["肝細胞癌", "膽管癌", "合併肝細胞及膽管癌", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)

"""# carcinoma_type = "Pancreas"
"""



carcinoma_type = "胰臟癌"

def Type_Group(type_code):
    adenocarcinoma  = [8140, 8441, 8480, 8481, 8490, 8500, 8503, 8510, 8550, 8551, 8552] #腺癌
    nurv_coma = [8150, 8151, 8152, 8153, 8155, 8156, 8158, 8240, 8246, 8249] #神經內分泌瘤
    others = [8000, 8010, 8013, 8020, 8035, 8041, 8148, 8154, 8452, 8453, 8470] #其它
    labels = ["腺癌", "神經內分泌瘤"]
    hist = type_code//10

    if hist in adenocarcinoma:
        return labels[0]
    if hist in nurv_coma:
        return labels[1]
    if hist in others:
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["腺癌", "神經內分泌瘤", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution

file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "Lung"

carcinoma_type = "肺癌"

def Type_Group(type_code):
    squamous  = [8070,8071,8072,8083] #鱗狀細胞癌
    adenocarcinoma_with_behavior = [82503, 82533, 82573, 82532, 82563, 82502]  #腺癌
    adenocarcinoma  = [8140, 8551, 8200, 8480, 8144, 8333, 8265, 8254, 8260, 8230] #腺癌
    small_cell = [8002, 8041, 8249, 8045, 8246, 8240] #小細胞癌
    # others = [] #其它
    labels = ["鱗狀細胞癌", "腺癌", "小細胞癌"]
    if type_code in adenocarcinoma_with_behavior:
        return labels[1]

    hist = type_code//10
    if hist in squamous:
        return labels[0]
    if hist in adenocarcinoma:
        return labels[1]
    if hist in small_cell:
        return labels[2]
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌", "腺癌", "小細胞癌", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution


file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "Breast"

carcinoma_type = "乳癌"

# Carcinoma_NST = [8500, 8035, 8201/3, 8022/3, 8230/3]
# Mucinous_ = [8480, 8430]
# Lobular [8520]
# Phyllodes_tumor = [9020/3]
# Papillary_ = [8503, 8504/3, 8504/3, 8507, 8509/3]
# Paget_nipple_ = [8540, 8543/3, 8541/3]

def Type_Group(type_code):
    Carcinoma_NST = [8500, 8035, 8201, 8022, 8230]
    Mucinous_ = [8480, 8430]
    Lobular = [8520]
    Phyllodes_tumor = [9020]
    Papillary_ = [8503, 8504, 8504, 8507, 8509]
    Paget_nipple_ = [8540, 8543, 8541]
    labels = ["侵襲性癌,非特定性", "黏液性癌", "侵襲性小葉癌", "惡性葉狀腫瘤", "侵襲性乳頭癌", "乳房Paget氏症"]

    if ((type_code%10) == 3):
        hist = type_code//10
        if hist in Carcinoma_NST:
            return labels[0]
        if hist in Mucinous_:
            return labels[1]
        if hist in Lobular:
            return labels[2]
        if hist in Phyllodes_tumor:
            return labels[3]
        if hist in Papillary_:
            return labels[4]
        if hist in Paget_nipple_:
            return labels[5]
        return "其他"
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["侵襲性癌,非特定性", "黏液性癌", "侵襲性小葉癌", "惡性葉狀腫瘤", "侵襲性乳頭癌", "乳房Paget氏症", "其他", "其他（carcinoma_in_situ）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（carcinoma_in_situ）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution



# def Type_Group(type_code):
#     bins = [85003, 85203, 82013, 85093, 84803, 85403, 85203]
#     #85223,
#     labels = ["侵襲性癌非特定性", "侵襲性小葉癌", "侵襲性癌非特定性", "侵襲性乳頭癌", "黏液性癌", "乳房Paget氏症", "惡性葉狀腫瘤"]
#     ## "其他", "其他不計"
#     for i in range(len(bins)):
#         if type_code == bins[i]:
#             return labels[i]
#     if (type_code%10 == 3):
#         return "其他"
#     return "其他不計 carcinoma_in_situ"

# def Type_Report(df):
#     df_report = df[['hist/behavior', '個案分類']]
#     # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
#     df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
#     df_report = df_report.reset_index(drop=True)
#     df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
#     order = ["侵襲性癌非特定性", "侵襲性小葉癌", "侵襲性乳頭癌", "黏液性癌", "乳房Paget氏症", "惡性葉狀腫瘤", "其他", "其他不計 carcinoma_in_situ"]
#     _distribution = df_report['組織類型/性態'].value_counts()
#     _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
#     _distribution['percentage'] = 1
#     carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

#     for i in range(len(_distribution)):
#         _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
#     _distribution = _distribution[_distribution['組織類型/性態'] != '其他不計 carcinoma_in_situ']
#     new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
#     _distribution.loc[len(_distribution)] = new_row
#     new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
#     _distribution.loc[len(_distribution)] = new_row
#     return _distribution



file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "Cervix"

carcinoma_type = "子宮頸癌"



def Type_Group(type_code):
    squamous_with_behavior  = [80703, 80853, 80863] #鱗狀細胞癌
    adenocarcinoma_with_behavior = [81403, 84833, 84843, 84823, 83103, 91103]  #腺癌
    # others = [] #其它
    labels = ["鱗狀細胞癌", "腺癌"]
    if type_code in squamous_with_behavior:
        return labels[0]
    if type_code in adenocarcinoma_with_behavior:
        return labels[1]
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["鱗狀細胞癌", "腺癌", "其他", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution



def Define_FIGO_Stage_(df):
    df_FIGO_stage = df[['class', 'ostage_c']]
    df_FIGO_stage['class'] = df_FIGO_stage['class'].str.replace("'", '').astype(int)
    df_FIGO_stage = df_FIGO_stage[df_FIGO_stage['class'].isin([1, 2])]
    df_FIGO_stage = df_FIGO_stage.reset_index(drop=True)
    df_FIGO_stage['ostage_c'] = df_FIGO_stage['ostage_c'].str.replace("'", '')

    df_FIGO_stage['STAGE'] = None
    # for i in range (len(df_FIGO_stage)):
    #     if (df_FIGO_stage['ostage_c'][i] not in ['9999', '8888']):
    #         df_FIGO_stage['STAGE'][i] = df_FIGO_stage['ostage_c'][i]
    #     else:
    #         "不詳期別"
    return df_FIGO_stage


def FIGO_Distribution_tabel(order, df_FIGO_stage):
    _distribution = df_FIGO_stage['STAGE'].value_counts()
    _distribution = df_FIGO_stage.groupby('STAGE').size().reindex(order, fill_value=0).reset_index(name='count')
    sum_counts = _distribution.loc[_distribution['STAGE'].isin(['9999', '8888']), 'count'].sum()
    _distribution = _distribution[~_distribution['STAGE'].isin(['9999', '8888'])]
    new_row = pd.DataFrame([{'STAGE': '不詳期別', 'count': sum_counts}])
    _distribution = pd.concat([_distribution, new_row], ignore_index=True)
    _distribution.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_FIGO_stage.xlsx', sheet_name='FIGO_stage')
    return _distribution



file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "Corpus_Uteri"
carcinoma_type = "子宮體癌"


def Type_Group(type_code):
    endometrioid_adenocarcinoma_with_behavior  = [83803, 85703] #子宮內膜樣癌
    serous_with_behavior = [84413] #漿液性腺癌
    clear_cell_adenocarcinoma_with_behavior = [83103] #明亮細胞癌
    coma = [8933, 8800, 8890, 8898, 8900, 8921, 8930, 8931, 8935]
    # others = [] #其它
    labels = ["子宮內膜樣癌", "漿液性腺癌", "明亮細胞癌", "上皮癌肉瘤"]
    if type_code in endometrioid_adenocarcinoma_with_behavior:
        return labels[0]
    if type_code in serous_with_behavior:
        return labels[1]
    if type_code in clear_cell_adenocarcinoma_with_behavior:
        return labels[2]
    hist = type_code//10
    if hist in coma:
        return labels[3]

    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["子宮內膜樣癌", "漿液性腺癌", "明亮細胞癌", "上皮癌肉瘤", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution



file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "Ovary"

carcinoma_type = "卵巢癌"

def Type_Group(type_code):
    endometrioid_adenocarcinoma_with_behavior  = [83803] #子宮內膜樣癌
    serous_with_behavior = [84413] #漿液性腺癌
    clear_cell_adenocarcinoma_with_behavior = [83103] #明亮細胞癌
    mucinous_with_behavior = [84803] #黏液性腺癌
    # others = [] #其它
    labels = ["子宮內膜樣癌", "漿液性腺癌", "明亮細胞癌", "黏液性腺癌"]
    if type_code in endometrioid_adenocarcinoma_with_behavior:
        return labels[0]
    if type_code in serous_with_behavior:
        return labels[1]
    if type_code in clear_cell_adenocarcinoma_with_behavior:
        return labels[2]
    if type_code in mucinous_with_behavior:
        return labels[3]

    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["子宮內膜樣癌", "漿液性腺癌", "明亮細胞癌", "黏液性腺癌", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution




file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "Prostate"

carcinoma_type = "攝護腺癌"


def Type_Group(type_code):
    adenocarcinoma = [8140, 8500] #腺癌
    adenocarcinoma_with_behavior = [82013, 82603, 82303, 85723,  82013, 82603, 82303, 84803, 84903] #腺癌
    urothelial_with_behavior = [81203] #非特定上皮細胞癌

    labels = ["腺癌", "非特定上皮細胞癌"]
    if type_code in adenocarcinoma_with_behavior:
        return labels[0]
    if type_code in urothelial_with_behavior:
        return labels[1]

    hist = type_code//10
    if hist in adenocarcinoma:
        return labels[0]

    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["腺癌", "非特定上皮細胞癌", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution



file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "Bladder"


carcinoma_type = "膀胱癌"


def Type_Group(type_code):
    adenocarcinoma = [8140, 8310, 8380, 8144, 8480] #腺癌
    urothelial = [8120] #泌尿上皮細胞癌
    urothelial_with_behavior = [80823, 80313, 80823, 81303, 81313, 80203, 81223] #泌尿上皮細胞癌

    labels = ["腺癌", "泌尿上皮細胞癌"]
    if type_code in urothelial_with_behavior:
        return labels[1]

    hist = type_code//10
    if hist in adenocarcinoma:
        return labels[0]
    if hist in urothelial:
        return labels[1]

    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["腺癌", "泌尿上皮細胞癌", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution


file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['0', '1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "淋巴結(外)惡性淋巴瘤"

carcinoma_type = "淋巴結(外)惡性淋巴瘤"


def Type_Group(type_code):
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["腺癌", "泌尿上皮細胞癌", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution


file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)


# # carcinoma_type = "淋巴結(內)惡性淋巴癌"


carcinoma_type = "淋巴結(內)惡性淋巴癌"

def Type_Group(type_code):
    Hodigkin = list(i for i in range(9650, 9668)) #[9650-9667] #何杰金氏

    non_Hodigkin_B_cell_MALT = [9689, 9699] #[9689\9699] #非何杰金氏
    non_Hodigkin_B_cell_FL = list(i for i in range(9690, 9699)) #[9689-9699] #非何杰金氏
    non_Hodigkin_B_cell_FL.append(9597)
    non_Hodigkin_B_cell_DL = [9596, 9651, 9654, 9670, 9673, 9675, 9680, 9684, 9738] #非何杰金氏
    non_Hodigkin_T_cell = list(i for i in range(9700, 9730))
    non_Hodigkin_T_cell.append(9827)

    non_Hodigkin_other_ = [9590, 9591, 9761, 9768, 9769, 9766, 9823, 9833, 9940]
    temp = list(i for i in range(9687, 9727))
    non_Hodigkin_other_ += temp
    temp = list(i for i in range(9735, 9738))
    non_Hodigkin_other_ += temp

    other_ = list(i for i in range(9750, 9761)) #[9750-9760]
    other_.append(9764)


    hist = type_code//10
    labels = ["何杰金氏淋巴瘤", "非何杰金氏淋巴瘤 [B 細胞淋巴瘤 (MALT 邊緣區/胃)]", "非何杰金氏淋巴瘤 [B 細胞淋巴瘤 (濾泡型)]", "非何杰金氏淋巴瘤 [B 細胞淋巴瘤 (瀰漫型)]", "非何杰金氏淋巴瘤 [T/NK 細胞淋巴瘤]", "非何杰金氏淋巴瘤 [其他]"]

    if hist in Hodigkin:
        # return str([labels[0], str(type_code)])
        return labels[0]

    if hist in non_Hodigkin_B_cell_MALT:
        return labels[1]

    if hist in non_Hodigkin_B_cell_FL:
        return labels[2]

    if hist in non_Hodigkin_B_cell_DL:
        return labels[3]

    if hist in non_Hodigkin_T_cell:
        return labels[4]

    if hist in non_Hodigkin_other_:
        return labels[5]

    if hist in other_:
        return "其他"

    return "其他（待確認）"


def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["何杰金氏淋巴瘤", "非何杰金氏淋巴瘤 [B 細胞淋巴瘤 (MALT 邊緣區/胃)]", "非何杰金氏淋巴瘤 [B 細胞淋巴瘤 (濾泡型)]", "非何杰金氏淋巴瘤 [B 細胞淋巴瘤 (瀰漫型)]", "非何杰金氏淋巴瘤 [T/NK 細胞淋巴瘤]", "非何杰金氏淋巴瘤 [其他]", "其他", '其他（待確認）']
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution



file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report, hist_distribution_report, df_AJCC_stage= generate_and_save_reports(df, carcinoma_type)

order = ['1', '2', '3', '4','BBB', '888', '999']
stage_distribution = Distribution_tabel(order, df_AJCC_stage)



# hist_distribution_report

H_ = int(hist_distribution_report['count'][0])
NH_ = int(hist_distribution_report['count'][len(hist_distribution_report)-2]) - H_ - int(hist_distribution_report['count'][len(hist_distribution_report)-3])
# H_, NH_

column_list = ["何杰金氏淋巴瘤", "非何杰金氏淋巴瘤"]
index_list = ["結內淋巴癌組織碼型態分佈", "結內淋巴癌組織碼型態分佈百分比"]
total = H_ + NH_
data = {
    '何杰金氏淋巴瘤': [H_, round((H_/total), 4)],
    '非何杰金氏淋巴瘤': [NH_, round((NH_/total), 4)],
}
df = pd.DataFrame(data, index=index_list, columns=column_list)

#Save file


df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_結內淋巴癌組織碼型態.xlsx', sheet_name='結內淋巴癌組織碼型態')



B_ = hist_distribution_report.loc[1:3, 'count'].sum()
T_ = hist_distribution_report.loc[4, 'count']
# B_, T_

column_list = ["B 細胞", "T 細胞"]
index_list = ["結內非何杰金氏淋巴組織型態分佈", "結內非何杰金氏淋巴組織型態分佈百分比"]
total = B_+ T_
data = {
    'B 細胞': [B_, round((B_/total), 4)],
    'T 細胞': [T_, round((T_/total), 4)],
}
df = pd.DataFrame(data, index=index_list, columns=column_list)
# df

#Save file
df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_結內非何杰金氏淋巴組織型態.xlsx', sheet_name='結內非何杰金氏淋巴組織型態')


FL_ = hist_distribution_report.loc[2, 'count'].sum()
DL_ = hist_distribution_report.loc[3, 'count']
# FL_, DL_



column_list = ["濾泡型", "瀰漫型"]
index_list = ["結內淋巴癌 B cell 組織型態分佈", "結內淋巴癌 B cell 組織型態分佈百分比"]
total = FL_+ DL_
data = {
    "濾泡型" : [FL_, round((FL_/total), 4)],
    "瀰漫型" : [DL_, round((DL_/total), 4)],
}
df = pd.DataFrame(data, index=index_list, columns=column_list)
# df



#Save file


df.to_excel(f'/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}Report/{year}_{carcinoma_type}_結內淋巴癌 B cell 組織型態.xlsx', sheet_name='結內淋巴癌 B cell 組織型態')


# # carcinoma_type = "白血病"

carcinoma_type = "白血病"


def Type_Group(type_code):
    return "其他（待確認）"

def Type_Report(df):
    df_report = df[['hist/behavior', '個案分類']]
    # df_report['hist/behavior'] = df_report['hist/behavior'].str.replace("'", '').astype(int)
    df_report = df_report[df_report['個案分類'].isin(['class1', 'class2'])]
    df_report = df_report.reset_index(drop=True)
    df_report['組織類型/性態'] = df_report['hist/behavior'].apply(Type_Group)
    order = ["腺癌", "泌尿上皮細胞癌", "其他（待確認）"]
    _distribution = df_report['組織類型/性態'].value_counts()
    _distribution = df_report.groupby('組織類型/性態').size().reindex(order, fill_value=0).reset_index(name='count')
    _distribution['percentage'] = 1
    carcinoma_in_situ = len(df_report)-_distribution['count'][len(_distribution)-1]

    for i in range(len(_distribution)):
        _distribution['percentage'][i] = (_distribution['count'][i]/carcinoma_in_situ).round(3)
    _distribution = _distribution[_distribution['組織類型/性態'] != '其他（待確認）']
    new_row = {'組織類型/性態': '合計', 'count': carcinoma_in_situ, 'percentage': 1.00}
    _distribution.loc[len(_distribution)] = new_row
    new_row = {'組織類型/性態': '總數(含原位癌)', 'count': len(df_report), 'percentage': None}
    _distribution.loc[len(_distribution)] = new_row

    return _distribution


file_path = f"/content/drive/My Drive/自動化癌症年報/{main_file_name}/output{year}/{year}{carcinoma_type}/{year}{carcinoma_type}.csv"
df = pd.read_csv(file_path)
age_gender_report, class_gender_report = generate_and_save_reports(df, carcinoma_type)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
目錄 '/content/drive/My Drive/自動化癌症年報/2023Report/output2021' 已成功建立！
其他（待確認）        241
乳癌             238
肺癌             164
結腸癌            157
攝護腺癌           109
肝癌              96
鼻咽癌             78
直腸癌             70
胃癌              69
膀胱癌             59
口腔癌             57
子宮頸癌            48
子宮體癌            40
白血病             38
胰臟癌             28
食道癌             24
卵巢癌             24
淋巴結(內)惡性淋巴癌     23
口咽癌             18
淋巴結(外)惡性淋巴瘤     14
喉癌               9
下咽癌              7
肛門癌              2
Name: Site, dtype: int64


<ipython-input-5-ba71711e4880>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_report['年齡區間'] = df_report['診斷年齡'].apply(Age_Group)
<ipython-input-5-ba71711e4880>:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_AJCC_stage['class'] = df_AJCC_stage['class'].str.replace("'", '').astype(int)
<ipython-input-5-ba71711e4880>:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u